# 債券プロジェクト

In [1]:
import pandas as pd
import datetime
import numpy as np

$t'_0 = \mbox{2021/11/05}$（現在）<br>
$t_0  = \mbox{2021/08/15}$（現在の直前にクーポンが支払われた時間）<br>
とする。

In [2]:
now = datetime.date(2021, 11, 5)
t0 = datetime.date(2021, 8, 15)

債務と債券情報の入力

In [3]:
bond_data = pd.DataFrame(data=[
    [6.625, datetime.date(2022, 2, 15), 100.00],
    [9.125, datetime.date(2022, 2, 15), 100.69],
    [7.875, datetime.date(2022, 8, 15), 100.75],
    [8.250, datetime.date(2022, 8, 15), 101.03],
    [8.250, datetime.date(2023, 2, 15), 101.22],
    [8.375, datetime.date(2023, 2, 15), 101.38],
    [8.000, datetime.date(2023, 8, 15), 100.81],
    [8.750, datetime.date(2023, 8, 15), 102.03],
    [6.875, datetime.date(2024, 2, 15),  98.16],
    [8.875, datetime.date(2024, 2, 15), 102.28],
    [6.875, datetime.date(2024, 8, 15),  97.41],
    [8.625, datetime.date(2024, 8, 15), 101.72],
    [7.750, datetime.date(2025, 2, 15),  99.16],
    [11.25, datetime.date(2025, 2, 15), 109.13],
    [8.500, datetime.date(2025, 8, 15), 101.41],
    [10.50, datetime.date(2025, 8, 15), 107.84],
    [7.875, datetime.date(2026, 2, 15),  99.41],
    [8.875, datetime.date(2026, 2, 15), 103.00],
    ],
    columns=['クーポン', '満期日', '価格'])
# 経過利息を価格に加える。（詳しくは問4-4参照）
bond_data['価格'] = bond_data['価格'] + bond_data['クーポン']*(now - t0).days/365

debt_data = pd.DataFrame(data=[
    [datetime.date(2022, 2, 15),  2000],
    [datetime.date(2022, 8, 15), 20000],
    [datetime.date(2023, 2, 15),     0],
    [datetime.date(2023, 8, 15), 25000],
    [datetime.date(2024, 2, 15),  1000],
    [datetime.date(2024, 8, 15),     0],
    [datetime.date(2025, 2, 15), 20000],
    [datetime.date(2025, 8, 15),  1000],
    [datetime.date(2026, 2, 15), 15000],
    ],
    columns=['支払日', '債務価格'])

In [4]:
bond_data.head()

,クーポン,満期日,価格
0,6.625,2022-02-15,101.488356
1,9.125,2022-02-15,102.740000
2,7.875,2022-08-15,102.519178
3,8.250,2022-08-15,102.883425
4,8.250,2023-02-15,103.073425


In [5]:
debt_data.head()

,支払日,債務価格
0,2022-02-15,2000
1,2022-08-15,20000
2,2023-02-15,0
3,2023-08-15,25000
4,2024-02-15,1000


スポットレートカーブを問4-4の結果で定義する。

In [6]:
def r(t):
    alpha = [3.9725e-02, 1.2560e-02, 1.5246e-03, -9.4998e-04, 8.3856e-05]
    return alpha[0] + alpha[1]*t + alpha[2]*t**2 + alpha[3]*t**3 + alpha[4]*t**4

まずはそれぞれの債券のキャッシュ・フローを書き下す（教科書の表5.3のイメージ）

In [7]:
data = [[now] + list(-bond_data['価格'])]
for i, t in enumerate(debt_data['支払日']):
    datum = [t]
    for j in bond_data.itertuples():
        datum.append(j.クーポン if j.満期日>t else j.クーポン+100 if j.満期日==t else 0)
    data.append(datum)

cash_flow = pd.DataFrame(data, columns=['年']+['実在債券{0}'.format(i+1) for i in range(len(bond_data))]).set_index('年')
cash_flow

,実在債券1,実在債券2,実在債券3,実在債券4,実在債券5,実在債券6,実在債券7,実在債券8,実在債券9,実在債券10,実在債券11,実在債券12,実在債券13,実在債券14,実在債券15,実在債券16,実在債券17,実在債券18
年,,,,,,,,,,,,,,,,,,
2021-11-05,-101.488356,-102.740,-102.519178,-102.883425,-103.073425,-103.261507,-102.60726,-103.995753,-99.704521,-104.273836,-98.954521,-103.657671,-100.901096,-111.657397,-103.319589,-110.198904,-101.179178,-104.993836
2022-02-15,106.625000,109.125,7.875000,8.250000,8.250000,8.375000,8.00000,8.750000,6.875000,8.875000,6.875000,8.625000,7.750000,11.250000,8.500000,10.500000,7.875000,8.875000
2022-08-15,0.000000,0.000,107.875000,108.250000,8.250000,8.375000,8.00000,8.750000,6.875000,8.875000,6.875000,8.625000,7.750000,11.250000,8.500000,10.500000,7.875000,8.875000
2023-02-15,0.000000,0.000,0.000000,0.000000,108.250000,108.375000,8.00000,8.750000,6.875000,8.875000,6.875000,8.625000,7.750000,11.250000,8.500000,10.500000,7.875000,8.875000
2023-08-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,108.00000,108.750000,6.875000,8.875000,6.875000,8.625000,7.750000,11.250000,8.500000,10.500000,7.875000,8.875000
2024-02-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,106.875000,108.875000,6.875000,8.625000,7.750000,11.250000,8.500000,10.500000,7.875000,8.875000
2024-08-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,106.875000,108.625000,7.750000,11.250000,8.500000,10.500000,7.875000,8.875000
2025-02-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,107.750000,111.250000,8.500000,10.500000,7.875000,8.875000
2025-08-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,108.500000,110.500000,7.875000,8.875000


## (a)簡単なキャッシュ・フロー・マッチング
ここに、余剰資金を運用するための人工的な債券のキャッシュ・フローを追加する。<br>
 今回は金利が$0$なので、例えば$i$期間後から$i+1$期間後までの間の再投資を表現するキャッシュ・フローは
\begin{align}
(0,0,\cdots,0,-1,1,0,\cdots,0 )
\end{align}
となる。($i=0,1,2,\cdots,n-1$)

In [8]:
for i in range(len(cash_flow)-1):
    datum = [0]*(len(debt_data)+1)
    datum[i] = -1
    datum[i+1] = 1
    cash_flow['人工債券{0}'.format(i+1)]=datum
cash_flow

,実在債券1,実在債券2,実在債券3,実在債券4,実在債券5,実在債券6,実在債券7,実在債券8,実在債券9,実在債券10,...,実在債券18,人工債券1,人工債券2,人工債券3,人工債券4,人工債券5,人工債券6,人工債券7,人工債券8,人工債券9
年,,,,,,,,,,,,,,,,,,,,,
2021-11-05,-101.488356,-102.740,-102.519178,-102.883425,-103.073425,-103.261507,-102.60726,-103.995753,-99.704521,-104.273836,...,-104.993836,-1,0,0,0,0,0,0,0,0
2022-02-15,106.625000,109.125,7.875000,8.250000,8.250000,8.375000,8.00000,8.750000,6.875000,8.875000,...,8.875000,1,-1,0,0,0,0,0,0,0
2022-08-15,0.000000,0.000,107.875000,108.250000,8.250000,8.375000,8.00000,8.750000,6.875000,8.875000,...,8.875000,0,1,-1,0,0,0,0,0,0
2023-02-15,0.000000,0.000,0.000000,0.000000,108.250000,108.375000,8.00000,8.750000,6.875000,8.875000,...,8.875000,0,0,1,-1,0,0,0,0,0
2023-08-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,108.00000,108.750000,6.875000,8.875000,...,8.875000,0,0,0,1,-1,0,0,0,0
2024-02-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,106.875000,108.875000,...,8.875000,0,0,0,0,1,-1,0,0,0
2024-08-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,8.875000,0,0,0,0,0,1,-1,0,0
2025-02-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,8.875000,0,0,0,0,0,0,1,-1,0
2025-08-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,8.875000,0,0,0,0,0,0,0,1,-1


利用可能な債券の数を$m$、$j$番目の債券の価格を$p_j$とする。<br>
債務の生じる期間を$n$期間とし、$i$期間後の債務を$y_i$、$j$番目の債券の$i$期間後のクーポン(とあれば満期返戻価格)を$c_{ij}$とする。<br>
$j$番目の債券の保有量を$x_j$とした時、
\begin{align}
\min_{\bf x}&\sum_{j=1}^mp_jx_j\\
s.t.\ \sum_{j=1}^mc_{ij}x_j\geq& y_i\ \ (i=1,2,\cdots,n)\\
x_j\geq& 0\ \ (j=1,2,\cdots,m)
\end{align}

In [9]:
from pulp import *
m = LpProblem() # 最小化問題として定式化
x = [LpVariable('x{0}'.format(i+1), lowBound=0) for i in range(len(cash_flow.columns))] # 正の変数xの設定
m += - (np.array(cash_flow[cash_flow.index==now])[0]*x).sum()  # 目的関数
for i, t in enumerate(debt_data['支払日']):
    # i期間後に生じる債務
    y = debt_data['債務価格'][i]
    # それぞれの債券でi期間後に得られるリターン
    c = np.array(cash_flow[cash_flow.index==t])[0]
    # 制約条件
    m += (c*x).sum() >= y

In [10]:
m.solve() # ソルバーの実行

1

18

In [11]:
ans_a = pd.DataFrame(data=[value(x[i]) for i in range(len(bond_data.index))], columns=['(a)'], index=cash_flow.columns[:len(bond_data.index)])
ans_a.loc['総費用'] = sum([value(x[i])*bond_data['価格'][i] for i in range(len(bond_data))])
ans_a

,(a)
実在債券1,0.000000
実在債券2,0.000000
実在債券3,0.000000
実在債券4,117.899790
実在債券5,0.000000
実在債券6,0.000000
実在債券7,0.000000
実在債券8,167.353260
実在債券9,0.000000
実在債券10,0.000000


これが費用最小のポートフォリオとなる。

## (b)複雑なキャッシュ・フロー・マッチング

余剰資金を運用するための人工的な債券のキャッシュ・フローを追加する。<br>
連続金利でのフォワードレートは
\begin{align}
f_{t_1,t_2} = \frac{r(t_2)t_2-r(t_1)t_1}{t_2-t_1}
\end{align}
である。ただし、現在(2021/11/5)から$i$回目の配当支払日までの時間を$t_i$年とする。<br>
なので、例えば$i$期間後から$i+1$期間後までの間の再投資を表現するキャッシュ・フローは
\begin{align}
(0,0,\cdots,0,-1,1+f_{t_i,t_{i+1}},0,\cdots,0 )
\end{align}

In [12]:
t = np.array([(cash_flow.index[i] - cash_flow.index[0]).days/365 for i in range(len(cash_flow))])
for i in range(len(cash_flow)-1):
    datum = [0]*(len(debt_data)+1)
    datum[i] = -1
    datum[i+1] = 1 + (r(t[i+1])*t[i+1] - r(t[i])*t[i])/(t[i+1] - t[i])
    cash_flow['人工債券{0}'.format(i+1)]=datum
cash_flow

,実在債券1,実在債券2,実在債券3,実在債券4,実在債券5,実在債券6,実在債券7,実在債券8,実在債券9,実在債券10,...,実在債券18,人工債券1,人工債券2,人工債券3,人工債券4,人工債券5,人工債券6,人工債券7,人工債券8,人工債券9
年,,,,,,,,,,,,,,,,,,,,,
2021-11-05,-101.488356,-102.740,-102.519178,-102.883425,-103.073425,-103.261507,-102.60726,-103.995753,-99.704521,-104.273836,...,-104.993836,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-02-15,106.625000,109.125,7.875000,8.250000,8.250000,8.375000,8.00000,8.750000,6.875000,8.875000,...,8.875000,1.043334,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2022-08-15,0.000000,0.000,107.875000,108.250000,8.250000,8.375000,8.00000,8.750000,6.875000,8.875000,...,8.875000,0.000000,1.053706,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-02-15,0.000000,0.000,0.000000,0.000000,108.250000,108.375000,8.00000,8.750000,6.875000,8.875000,...,8.875000,0.000000,0.000000,1.066613,-1.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2023-08-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,108.00000,108.750000,6.875000,8.875000,...,8.875000,0.000000,0.000000,0.000000,1.077363,-1.000000,0.000000,0.000000,0.000000,0.000000
2024-02-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,106.875000,108.875000,...,8.875000,0.000000,0.000000,0.000000,0.000000,1.084698,-1.000000,0.000000,0.000000,0.000000
2024-08-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,8.875000,0.000000,0.000000,0.000000,0.000000,0.000000,1.088025,-1.000000,0.000000,0.000000
2025-02-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,8.875000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.087318,-1.000000,0.000000
2025-08-15,0.000000,0.000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,...,8.875000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.083259,-1.000000


この状態で(a)と同じ最適化を行う。以下再掲

利用可能な債券の数を$m$、$j$番目の債券の価格を$p_j$とする。<br>
債務の生じる期間を$n$期間とし、$i$期間後の債務を$y_i$、$j$番目の債券の$i$期間後のクーポン(とあれば満期返戻価格)を$c_{ij}$とする。<br>
$j$番目の債券の保有量を$x_j$とした時、
\begin{align}
\min_{\bf x}&\sum_{j=1}^mp_jx_j\\
s.t.\ \sum_{j=1}^mc_{ij}x_j\geq& y_i\ \ (i=1,2,\cdots,n)\\
x_j\geq& 0\ \ (j=1,2,\cdots,m)
\end{align}

In [13]:
m = LpProblem() # 最小化問題として定式化
x = [LpVariable('x{0}'.format(i+1), lowBound=0) for i in range(len(cash_flow.columns))] # 正の変数xの設定
m += - (np.array(cash_flow[cash_flow.index==now])[0]*x).sum()  # 目的関数
for i, t in enumerate(debt_data['支払日']):
    # i期間後に生じる債務
    y = debt_data['債務価格'][i]
    # それぞれの債券でi期間後に得られるリターン
    c = np.array(cash_flow[cash_flow.index==t])[0]
    # 制約条件
    m += (c*x).sum() >= y

In [14]:
m.solve() # ソルバーの実行


1

In [15]:
ans_b = pd.DataFrame(data=[value(x[i]) for i in range(len(bond_data.index))], columns=['(b)'], index=cash_flow.columns[:len(bond_data.index)])
ans_b.loc['総費用'] = sum([value(x[i])*bond_data['価格'][i] for i in range(len(bond_data))])
ans_b

,(b)
実在債券1,0.000000
実在債券2,0.000000
実在債券3,0.000000
実在債券4,113.808320
実在債券5,0.000000
実在債券6,0.000000
実在債券7,0.000000
実在債券8,161.003330
実在債券9,0.000000
実在債券10,0.000000


これが費用最小のポートフォリオとなる。

## (c)デュレーション・マッチング
期間$i$における$j$番目の債券のキャッシュフローを$c_{ij}$、債務のキャッシュフローを$y_i$とする。<br>
また現在から$i$期間後までの時間を$t_i$年とする。

In [16]:
# 時間
t = np.array([(cash_flow.index[i] - cash_flow.index[0]).days/365 for i in range(len(cash_flow))])
# 債券のキャッシュフローを格納した行列
c = np.array(cash_flow.loc[:, cash_flow.columns.str.contains('実在')])
# 債務のキャッシュフローを格納した行列
y = np.append(0, np.array(debt_data['債務価格']))

$j$番目の債券の現在価値を$PV_j$、債務の現在価値を$PV$とすると、
\begin{align}
PV_j = \sum_{i=1}^nc_{ij}e^{-r(t_i)t_i}\\
PV = \sum_{i=1}^ny_ie^{-r(t_i)t_i}
\end{align}

In [17]:
# 債券の現在価値(bond)
PVb = (np.exp(-r(t[1:])*t[1:])*c[1:].T).sum(axis=1)
# 債務の現在価値(debt)
PVd = (np.exp(-r(t[1:])*t[1:])*y[1:]).sum()

また、期間構造のパラメータ$\alpha_k$の変化に対して守る場合、$j$番目の債券のデュレーションを$D^{(k)}_j$、債務のデュレーションを$D^{(k)}$とすると、
\begin{align}
D^{(k)}_j &= -\frac{1}{PV_j}\frac{d}{d\alpha_k}PV_j\\
&=-\frac{1}{PV_j}\sum_{i=1}^nc_{ij}\frac{d}{d\alpha_k}e^{-r(t_i)t_i}\\
&=-\frac{1}{PV_j}\sum_{i=1}^nc_{ij}e^{-r(t_i)t_i}(-t_i^kt_i)\\
&= \frac{1}{PV_j}\sum_{i=1}^nt_i^{k+1}c_{ij}e^{-r(t_i)t_i}\\
D^{(k)} &= -\frac{1}{PV}\frac{d}{d\alpha_k}PV\\
 &= -\frac{1}{PV}\frac{d}{d\alpha_k}\sum_{i=1}^ny_ie^{-r(t_i)t_i}\\
 &= \frac{1}{PV}\sum_{i=1}^nt_i^{k+1}y_ie^{-r(t_i)t_i}\\
\end{align}

In [18]:
# 債券のデュレーション
Db = np.empty((5, len(bond_data)))
for k in range(5):
    Db[k] = (t**(k+1)*c.T*np.exp(-r(t)*t)).sum(axis=1)/PVb
# 債務のデュレーション
Dd = np.empty((5))
for k in range(5):
    Dd[k] = (t**(k+1)*y*np.exp(-r(t)*t)).sum()/PVd

したがって$j$番目の債券の購入単位を$x_j$とすると、ケース$l$の場合におけるイミュナイズの方程式は、
\begin{align}
\sum_{j=1}^mx_jPV_j &= PV\\
\sum_{j=1}^mx_jPV_jD^{(k)}_j &= PVD^{(k)}\ \ \ (k=0,1,\cdots,l-1)
\end{align}

これを制約として、初期の購入価格を最小にする最小問題を解けば良い。
\begin{align}
\min_{\bf x}\sum_{j=1}^mx_jp_j
\end{align}

In [19]:
ans_df = pd.DataFrame(data=[], columns=['ケース{0}'.format(l) for l in range(1,6)], index=ans_a.index)
for l in range(1,6):
    m = LpProblem() # 最小化問題として定式化
    x = [LpVariable('x{0}'.format(i+1), lowBound=0) for i in range(len(bond_data))] # 正の変数xの設定
    m += (bond_data['価格']*x).sum() # 目的関数
    m += (PVb*x).sum() == PVd # 現在価値の制約
    for k in range(l):
        m += (Db[k]*PVb*x).sum() == PVd*Dd[k] # デュレーションの制約
    m.solve() # ソルバーの実行

    ans_df['ケース{0}'.format(l)] = [value(x[i]) for i in range(len(x))]+[([value(x[i]) for i in range(len(x))]*bond_data['価格']).sum()]

In [20]:
ans_df

,ケース1,ケース2,ケース3,ケース4,ケース5
実在債券1,0.000000,0.000000,0.000000,0.000000,0.000000
実在債券2,98.393866,104.526970,0.000000,2.712146,0.000000
実在債券3,0.000000,0.000000,0.000000,0.000000,26.812528
実在債券4,0.000000,0.000000,0.000000,0.000000,0.000000
実在債券5,0.000000,0.000000,0.000000,0.000000,219.963900
実在債券6,0.000000,0.000000,220.656520,204.228400,0.000000
実在債券7,0.000000,0.000000,0.000000,0.000000,0.000000
実在債券8,0.000000,0.000000,18.123733,36.189765,0.000000
実在債券9,0.000000,0.000000,0.000000,0.000000,0.000000
実在債券10,0.000000,0.000000,0.000000,0.000000,0.000000


これがそれぞれのパターンにおいてイミュナイズされたポートフォリオの構成である。

三つのケースを並べて表示する

In [21]:
pd.concat([ans_a, ans_b, ans_df], axis = 1, join='inner')

,(a),(b),ケース1,ケース2,ケース3,ケース4,ケース5
実在債券1,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
実在債券2,0.000000,0.000000,98.393866,104.526970,0.000000,2.712146,0.000000
実在債券3,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,26.812528
実在債券4,117.899790,113.808320,0.000000,0.000000,0.000000,0.000000,0.000000
実在債券5,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,219.963900
実在債券6,0.000000,0.000000,0.000000,0.000000,220.656520,204.228400,0.000000
実在債券7,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
実在債券8,167.353260,161.003330,0.000000,0.000000,18.123733,36.189765,0.000000
実在債券9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
実在債券10,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
